In [2]:
import graph_tool.all as gt
from hSBM_Topicmodel.sbmtm import sbmtm

import pandas as pd
from collections import Counter
import nltk
import string
import regex as re
import numpy as np
import ast

from spacy.lang.da.stop_words import STOP_WORDS

## Preprocessing   
Later half not needed now since we already cleaned and lemmatized the dataset elsewhere. 

In [59]:
df = pd.read_csv("lemma_all.csv", parse_dates=['tweet_created_at'], compression='zip')
df.head()

,user_screen_name,tweet_id,tweet_created_at,tweet_full_text,tweet_text_lemma,tweet_text_lemma_reduced
0,AGrinsted,8.183690e+17,2017-01-09 08:09:20+00:00,RT @madsnyvold: Nogle sukker over stormflod og...,sukke stormflod høj vandstand bekæmpe natur kraft,sukke stormflod vandstand bekæmpe natur kraft
1,AGrinsted,8.184467e+17,2017-01-09 13:17:52+00:00,@COWIdk konkluderer at @kystdk's højvandsstati...,konkludere s højvandsstatistik underestimerer ...,konkludere s underestimerer niveau 100års stor...
2,AGrinsted,8.184475e+17,2017-01-09 13:21:08+00:00,Forskellen er at @Cowidk inkluderer historiske...,forskel inkludere historisk kilde bruge måle data,forskel inkludere kilde bruge måle data
3,AGrinsted,8.184543e+17,2017-01-09 13:48:11+00:00,@kristianpagh Kender du denne rapport for KBH?...,kende rapport kbh,kende rapport
4,AGrinsted,8.185035e+17,2017-01-09 17:03:43+00:00,@kristianpagh er den for høj i køge? -Kbh er j...,høj køge kbh fokus,fokus


In [60]:
# removing duplicates from 
df.drop_duplicates(subset='tweet_id')

,user_screen_name,tweet_id,tweet_created_at,tweet_full_text,tweet_text_lemma,tweet_text_lemma_reduced
0,AGrinsted,8.183690e+17,2017-01-09 08:09:20+00:00,RT @madsnyvold: Nogle sukker over stormflod og...,sukke stormflod høj vandstand bekæmpe natur kraft,sukke stormflod vandstand bekæmpe natur kraft
1,AGrinsted,8.184467e+17,2017-01-09 13:17:52+00:00,@COWIdk konkluderer at @kystdk's højvandsstati...,konkludere s højvandsstatistik underestimerer ...,konkludere s underestimerer niveau 100års stor...
2,AGrinsted,8.184475e+17,2017-01-09 13:21:08+00:00,Forskellen er at @Cowidk inkluderer historiske...,forskel inkludere historisk kilde bruge måle data,forskel inkludere kilde bruge måle data
3,AGrinsted,8.184543e+17,2017-01-09 13:48:11+00:00,@kristianpagh Kender du denne rapport for KBH?...,kende rapport kbh,kende rapport
4,AGrinsted,8.185035e+17,2017-01-09 17:03:43+00:00,@kristianpagh er den for høj i køge? -Kbh er j...,høj køge kbh fokus,fokus
...,...,...,...,...,...,...
333287,zeniastampe,1.371068e+18,2021-03-14 11:55:43+00:00,Vi lancerer i dag stort udspil for plantebaser...,lancere dag stor udspil plantebaserede fødevar...,lancere dag udspil fødevare investering plante...
333288,zeniastampe,1.371423e+18,2021-03-15 11:26:58+00:00,Aftale om sommerens festivaler præsenteres om ...,aftale sommer festival præsentere fest tilfred...,aftale sommer festival præsentere fest forvent...
333289,zeniastampe,1.387136e+18,2021-04-27 20:05:33+00:00,"Tror I, at vi er dumme? Regeringen reklamerer ...",tro dumme regering reklamere reduktion landbru...,tro regering reklamere reduktion landbrug mio ...
333290,zeniastampe,1.387286e+18,2021-04-28 06:01:04+00:00,"@hansersej Det er vi fuldt bevidste om, hvilke...",fuld bevidst hvilken utilfredshed regering akt...,utilfredshed regering


In [61]:
# only take tweets after 5 june 2019
after_election = df.loc[df.tweet_created_at > '05-06-2020']

In [62]:
len(after_election.tweet_created_at.dt.year)

124865

In [66]:
print(after_election.shape)

# aggregating tweets for actors for each month
# creating new column with month and year to aggregate on
after_election.loc[:, 'year'] = after_election['tweet_created_at'].dt.year  #.astype(str)
after_election.loc[:, 'month'] = after_election['tweet_created_at'].dt.month  #.astype(str)

(124865, 8)


/Users/jeppefoldberg/anaconda3/envs/graph-tool/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [67]:
after_election.head()

,user_screen_name,tweet_id,tweet_created_at,tweet_full_text,tweet_text_lemma,tweet_text_lemma_reduced,year,month
122,AGrinsted,1.271022e+18,2020-06-11 10:10:57+00:00,En 2015 artikel om slavelandet danmark. Check ...,2015 artikel slavelandet danmark check tegning,artikel slavelandet tegning,2020,6
123,AGrinsted,1.273534e+18,2020-06-18 08:31:29+00:00,Jeg tør godt at installere smitte|stop app'en....,tø godte installere smitte stoppe app glad goo...,tø godte installere stoppe app google rette sl...,2020,6
124,AGrinsted,1.275477e+18,2020-06-23 17:11:13+00:00,RT @KlimaMin: DMI er en skattekiste med værdif...,dmi skattekiste værdifuld vejrdata dag stille ...,skattekiste vejrdata dag rådighed gøre bo,2020,6
125,AGrinsted,1.285559e+18,2020-07-21 12:55:58+00:00,@SDFE_RS @nrojbrohT Jeg har kunnet måle højden...,kunne måle højde søster flagstang downloade data,måle søster flagstang downloade data,2020,7
126,AGrinsted,1.295615e+18,2020-08-18 06:54:21+00:00,Her hvor jeg sidder er det kun 3 ud af 9 der b...,sidde 3 9 bære maske s tog dårlig billet ugyld...,sidde bære maske s tog billet maske,2020,8


In [68]:
after_election.dtypes

user_screen_name                         object
tweet_id                                float64
tweet_created_at            datetime64[ns, UTC]
tweet_full_text                          object
tweet_text_lemma                         object
tweet_text_lemma_reduced                 object
year                                      int64
month                                     int64
dtype: object

In [78]:
# removing nas from tweet_text_lemma  --> viewing them after_election.loc[after_election.tweet_text_lemma.isna()]
after_election = after_election.dropna(subset=['tweet_text_lemma'])

In [79]:
after_election.shape

(124472, 8)

In [80]:
# aggregating the dataframe
tweets_agg = after_election.groupby(['user_screen_name', 'year', 'month'], as_index = False).\
    agg({'tweet_text_lemma': ' '.join})

In [20]:
# not needed now! 
def preprocess(text, tokenizer=nltk.tokenize.casual.TweetTokenizer()):
    '''takes in text and removes urls, mentions, numbers and punctuation'''
    if type(text) == str:
        text = text.lower()
        text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text)
        text = re.sub(r'https?', '', text)  # removes the extra weird https left in the text. 
        text = re.sub(r'\.\.\.', '', text)  # we do not care if people use ...
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'[0-9]', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = tokenizer.tokenize(text)
    else: 
        return np.nan
    return text

In [132]:
tweets_agg

,user_screen_name,year,month,tweet_text_lemma
0,AGrinsted,2020,6,2015 artikel slavelandet danmark check tegning...
1,AGrinsted,2020,7,kunne måle højde søster flagstang downloade data
2,AGrinsted,2020,8,sidde 3 9 bære maske s tog dårlig billet ugyld...
3,AGrinsted,2020,9,se imponere overvælde captura
4,AGrinsted,2020,10,udvikling spædbørnsdødelighed dk sakke bagud i...
...,...,...,...,...
2546,zeniastampe,2020,12,tak jens tilfælles selvtak sparring uvurderlig...
2547,zeniastampe,2021,1,forhandle desværre præcis grund ekstrem kritis...
2548,zeniastampe,2021,2,godte forstå passe god støjbergs fortælling fy...
2549,zeniastampe,2021,3,vigtig mission fredag aften kæmpe sommer festi...


In [135]:
# df['preprocessed_text'] = df.tweet_full_text.apply(preprocess)
# now we just need to tokenize the tweets! 
tokenizer = nltk.tokenize.casual.TweetTokenizer()
tweets_agg.loc[:, 'tokens'] = tweets_agg.tweet_text_lemma.apply(lambda x: tokenizer.tokenize(x))

In [136]:
tweets_agg

,user_screen_name,year,month,tweet_text_lemma,tokens
0,AGrinsted,2020,6,2015 artikel slavelandet danmark check tegning...,"[2015, artikel, slavelandet, danmark, check, t..."
1,AGrinsted,2020,7,kunne måle højde søster flagstang downloade data,"[kunne, måle, højde, søster, flagstang, downlo..."
2,AGrinsted,2020,8,sidde 3 9 bære maske s tog dårlig billet ugyld...,"[sidde, 3, 9, bære, maske, s, tog, dårlig, bil..."
3,AGrinsted,2020,9,se imponere overvælde captura,"[se, imponere, overvælde, captura]"
4,AGrinsted,2020,10,udvikling spædbørnsdødelighed dk sakke bagud i...,"[udvikling, spædbørnsdødelighed, dk, sakke, ba..."
...,...,...,...,...,...
2546,zeniastampe,2020,12,tak jens tilfælles selvtak sparring uvurderlig...,"[tak, jens, tilfælles, selvtak, sparring, uvur..."
2547,zeniastampe,2021,1,forhandle desværre præcis grund ekstrem kritis...,"[forhandle, desværre, præcis, grund, ekstrem, ..."
2548,zeniastampe,2021,2,godte forstå passe god støjbergs fortælling fy...,"[godte, forstå, passe, god, støjbergs, fortæll..."
2549,zeniastampe,2021,3,vigtig mission fredag aften kæmpe sommer festi...,"[vigtig, mission, fredag, aften, kæmpe, sommer..."


## Taking the data that we need for the topic model

In [140]:
tokens = tweets_agg['tokens']
handles = tweets_agg['user_screen_name']

In [141]:
hsbm_df = pd.DataFrame({'handles': handles, 'tokens': tokens})

In [142]:
hsbm_df

,handles,tokens
0,AGrinsted,"[2015, artikel, slavelandet, danmark, check, t..."
1,AGrinsted,"[kunne, måle, højde, søster, flagstang, downlo..."
2,AGrinsted,"[sidde, 3, 9, bære, maske, s, tog, dårlig, bil..."
3,AGrinsted,"[se, imponere, overvælde, captura]"
4,AGrinsted,"[udvikling, spædbørnsdødelighed, dk, sakke, ba..."
...,...,...
2546,zeniastampe,"[tak, jens, tilfælles, selvtak, sparring, uvur..."
2547,zeniastampe,"[forhandle, desværre, præcis, grund, ekstrem, ..."
2548,zeniastampe,"[godte, forstå, passe, god, støjbergs, fortæll..."
2549,zeniastampe,"[vigtig, mission, fredag, aften, kæmpe, sommer..."


In [143]:
# save it to a csv if we need to run the model again
compression = dict(method='zip', archive_name= 'hsbm_df.csv')
hsbm_df.to_csv('hsbm_df.zip', compression=compression)

# and reading it in again
#hsbm_df_new = pd.read_csv('hsbm_df.zip', compression='zip', index_col=0)

## Getting the data in a format usable for the hSBM

In [118]:
def extract_dict(string):
    """Helper function to extract dict from string if string exists 
    else the function returns an empty dict (Thanks, Tweepy...)
    Here it is used to extract lists; that works?!"""
    try:
        out = ast.literal_eval(string)
    except:
        out = dict()
    return out

hsbm_df["tokens"] = hsbm_df.tokens.apply(lambda x: extract_dict(x))

In [157]:
# Remowing stopwords from the list imported in the beginning
docs = [[word for word in doc if word not in STOP_WORDS] for doc in hsbm_df.tokens]

In [158]:
# Remove infrequent words; Snorres solution
cutoff = 5
c = Counter()
for doc in docs:
    c.update(Counter(doc))
vocab = c.most_common(40000)
vocab = set([word for word, count in vocab if count > 1])
# remove words
docs = [[w for w in doc if w in vocab] for doc in docs]

# Training the hSBM model

In [159]:
# step 3
model = sbmtm()

## we have to create the document network from the corpus
model.make_graph(docs)

gt.seed_rng(42) ## seed for graph-tool's random number generator --> same results
model.fit()

In [161]:
model.save_graph(filename = 'graph_full_dataset_grouped.xml.gz')

In [160]:
model.topics()

{0: [('konsekvens', 0.0561629474314812),
  ('engang', 0.03032799161299985),
  ('alvor', 0.025460536168938147),
  ('årti', 0.02246517897259248),
  ('fejre', 0.01639958064999251),
  ('årsag', 0.01580050921072338),
  ('rose', 0.010858169836753032),
  ('skidt', 0.010633518047027108),
  ('tjek', 0.00980979481803205),
  ('insistere', 0.009585143028306126)],
 1: [('artikel', 0.6744847192608386),
  ('on', 0.15991471215351813),
  ('dvs', 0.10021321961620469),
  ('you', 0.06538734896943853)],
 2: [('danmark', 0.9441755404969345),
  ('ly', 0.03630203291384317),
  ('gigantisk', 0.01952242658922233)],
 3: [('ord', 0.11477946866133608),
  ('høre', 0.11258705184420945),
  ('vist', 0.0949187516120712),
  ('ude', 0.08576218725818932),
  ('slette', 0.08344080474593758),
  ('blotte', 0.07957183389218468),
  ('vildt', 0.0631931906112974),
  ('nævne', 0.04836213567191127),
  ('hørt', 0.043848336342532886),
  ('åbenbar', 0.038947639927779214)],
 4: [('tilslutte', 0.02271386430678466),
  ('tegning', 0.021238

In [162]:
def get_topic_df(topics):
  '''takes in model.topic and returns a df that can be used to save the topics 
  could (and probably should) be rewritten with list comprehensions!'''
  topic_nr = []
  words = []
  weights = []
  for topic in topics: 
    for word in topics[topic]:
      topic_nr.append('topic ' + str(topic))
      words.append(word[0])
      weights.append(word[1])

  return pd.DataFrame({'topic_nr': topic_nr,
                'words': words,
                'weights': weights})

In [164]:
topic_df = get_topic_df(model.topics())

In [165]:
topic_df.to_csv('topics_grouped_dataset.csv')

## Finding the green words in the topics from hSBM

In [5]:
# reading in the topics from the largest model! 
topic_df = pd.read_csv('topics_grouped_dataset.csv', index_col=0)

In [166]:
# We have 94 topics
topic_df.tail()

,topic_nr,words,weights
3180,topic 373,fredelig,0.064140
3181,topic 373,kulturel,0.055394
3182,topic 373,fordom,0.055394
3183,topic 373,forårsage,0.053450
3184,topic 373,sikandar,0.039845


In [184]:
temp = topic_df.loc[topic_df.topic_nr == 'topic 4'].reset_index()

In [185]:
temp.words

0     tilslutte
1       tegning
2         visse
3       frigive
4       tilhøre
5          bøje
6    gammeldags
7     tiltrække
8          råde
9       youtube
Name: words, dtype: object

In [195]:
def add_words(topic_df):
    '''Ought to be rewritten to handle exeptions or errors in input made by the user'''
    temp_words = []
    for i, word in enumerate(topic_df.words):
        print('\n', i, word, '\n')
    inp = input('Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:\n')
    indices = [int(i) for i in inp.split(' ') if i.isdigit()]
    for i in indices:
        temp_words.append(topic_df.words[i])
    return temp_words

In [196]:
def check_topics(n_topics, df):
    all_words = []
    for i in range(n_topics):
        # iterates over each topic nr and returns a df with that topic
        topic_df = df.loc[df.topic_nr == f'topic {i+1}'].reset_index()
        print(topic_df)
        inp = input('\n\nDoes this topic include a green word? y or n?\n')
        if inp == 'y':
            all_words += add_words(topic_df)
        elif inp == 'n':
            continue
        else:
            print('I did not understand that!')
    return all_words


all_words = check_topics(373, topic_df)

   index topic_nr    words   weights
0     10  topic 1  artikel  0.674485
1     11  topic 1       on  0.159915
2     12  topic 1      dvs  0.100213
3     13  topic 1      you  0.065387


Does this topic include a green word? y or n?
n
   index topic_nr      words   weights
0     14  topic 2    danmark  0.944176
1     15  topic 2         ly  0.036302
2     16  topic 2  gigantisk  0.019522


Does this topic include a green word? y or n?
n
   index topic_nr    words   weights
0     17  topic 3      ord  0.114779
1     18  topic 3     høre  0.112587
2     19  topic 3     vist  0.094919
3     20  topic 3      ude  0.085762
4     21  topic 3   slette  0.083441
5     22  topic 3   blotte  0.079572
6     23  topic 3    vildt  0.063193
7     24  topic 3    nævne  0.048362
8     25  topic 3     hørt  0.043848
9     26  topic 3  åbenbar  0.038948


Does this topic include a green word? y or n?
n
   index topic_nr       words   weights
0     27  topic 4   tilslutte  0.022714
1     28  topic 4     



Does this topic include a green word? y or n?
n
   index  topic_nr                words   weights
0    180  topic 21                 vært  0.005336
1    181  topic 21            tilhænger  0.003766
2    182  topic 21               brutal  0.003505
3    183  topic 21           splittelse  0.003348
4    184  topic 21              tavshed  0.003034
5    185  topic 21               talent  0.002825
6    186  topic 21  grænseoverskridende  0.002825
7    187  topic 21                 line  0.002511
8    188  topic 21          fornemmelse  0.002354
9    189  topic 21                 anse  0.002040


Does this topic include a green word? y or n?
n
   index  topic_nr      words   weights
0    190  topic 22      glæde  0.913941
1    191  topic 22        sam  0.030120
2    192  topic 22  værdifuld  0.029260
3    193  topic 22    snarest  0.026678


Does this topic include a green word? y or n?
n
   index  topic_nr            words   weights
0    194  topic 23         indenfor  0.153461
1    195



Does this topic include a green word? y or n?
n
   index  topic_nr       words   weights
0    336  topic 40      fremme  0.425658
1    337  topic 40  anbefaling  0.122570
2    338  topic 40       indgå  0.095120
3    339  topic 40     deltage  0.074342
4    340  topic 40    invitere  0.052802
5    341  topic 40    erfaring  0.049180
6    342  topic 40         hør  0.037362
7    343  topic 40         tæt  0.034884
8    344  topic 40       front  0.026496
9    345  topic 40          co  0.025925


Does this topic include a green word? y or n?
n
   index  topic_nr          words   weights
0    346  topic 41           2016  0.044717
1    347  topic 41  præsidentvalg  0.032004
2    348  topic 41       indsætte  0.029811
3    349  topic 41     washington  0.028496
4    350  topic 41         bruger  0.027181
5    351  topic 41     forfatning  0.016221
6    352  topic 41         harris  0.015783
7    353  topic 41          vægte  0.015783
8    354  topic 41          obama  0.015783
9    355 



Does this topic include a green word? y or n?
y

 0 hey 


 1 videnskabelig 


 2 svine 


 3 pensionskasse 


 4 carbon 


 5 klimaforsker 


 6 estimat 


 7 lt 


 8 2010 


 9 kursus 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
5
   index  topic_nr words   weights
0    486  topic 57    dk  0.921854
1    487  topic 57   dks  0.078146


Does this topic include a green word? y or n?
n
   index  topic_nr            words   weights
0    488  topic 58               fn  0.122860
1    489  topic 58   internationale  0.111830
2    490  topic 58             havn  0.049829
3    491  topic 58              150  0.044123
4    492  topic 58       enestående  0.019399
5    493  topic 58             regn  0.019019
6    494  topic 58  generalsekretær  0.018258
7    495  topic 58  udenrigspolitik  0.016736
8    496  topic 58       protestere  0.015215
9    497  topic 58        aflysning  0.014835


Does this topic include a green word? y



Does this topic include a green word? y or n?
y

 0 fix 


 1 inddragelse 


 2 overskygge 


 3 klimaborgertinget 


 4 borgerting 


 5 borgerinddragelse 


 6 borgertinget 


 7 undvige 


 8 kødforbrug 


 9 forskningscenter 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
3 8
   index  topic_nr   words   weights
0    635  topic 73  støtte  0.924595
1    636  topic 73  omfang  0.075405


Does this topic include a green word? y or n?
n
   index  topic_nr           words   weights
0    637  topic 74              kø  0.303821
1    638  topic 74            føde  0.207829
2    639  topic 74          klasse  0.184529
3    640  topic 74        overfald  0.067102
4    641  topic 74        kunstner  0.066170
5    642  topic 74        anbringe  0.053122
6    643  topic 74              42  0.046598
7    644  topic 74          jødisk  0.042870
8    645  topic 74  socialrådgiver  0.027959


Does this topic include a green word? y or n?

...



Does this topic include a green word? y or n?
n
   index   topic_nr             words   weights
0   2565  topic 303     statsminister  0.500650
1   2566  topic 303         borgerlig  0.157997
2   2567  topic 303          indkalde  0.106957
3   2568  topic 303   statsborgerskab  0.052991
4   2569  topic 303   justitsminister  0.048440
5   2570  topic 303  statsministerium  0.047139
6   2571  topic 303          hækkerup  0.033485
7   2572  topic 303        folkestyre  0.030559
8   2573  topic 303        spørgetime  0.021782


Does this topic include a green word? y or n?
n
   index   topic_nr          words   weights
0   2574  topic 304           vand  0.273229
1   2575  topic 304      forsvinde  0.228151
2   2576  topic 304    ministerium  0.167433
3   2577  topic 304         jesper  0.083717
4   2578  topic 304  miljøstyrelse  0.076357
5   2579  topic 304             øl  0.061638
6   2580  topic 304             36  0.047838
7   2581  topic 304            kat  0.035879
8   2582  topic



Does this topic include a green word? y or n?
n
   index   topic_nr            words   weights
0   2708  topic 319           fattig  0.320038
1   2709  topic 319             rige  0.199430
2   2710  topic 319     klimabistand  0.134853
3   2711  topic 319         klimatal  0.124406
4   2712  topic 319  klimatilpasning  0.090218
5   2713  topic 319     klimaudsatte  0.050332
6   2714  topic 319      klimastøtte  0.040836
7   2715  topic 319             care  0.039886


Does this topic include a green word? y or n?
y

 0 fattig 


 1 rige 


 2 klimabistand 


 3 klimatal 


 4 klimatilpasning 


 5 klimaudsatte 


 6 klimastøtte 


 7 care 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
2 3 4 5 6
   index   topic_nr        words   weights
0   2716  topic 320        mette  0.592427
1   2717  topic 320  frederiksen  0.407573


Does this topic include a green word? y or n?
n
   index   topic_nr                words   weights
0   

   index   topic_nr words  weights
0   2837  topic 335    sf      1.0


Does this topic include a green word? y or n?
n
   index   topic_nr           words   weights
0   2838  topic 336      alternativ  0.520206
1   2839  topic 336               å  0.127847
2   2840  topic 336             bus  0.102131
3   2841  topic 336   lynetteholmen  0.060985
4   2842  topic 336        gensidig  0.033799
5   2843  topic 336             mfl  0.030125
6   2844  topic 336         torsten  0.022777
7   2845  topic 336          tilmed  0.022043
8   2846  topic 336       franciska  0.018369
9   2847  topic 336  forsørgerpligt  0.014695


Does this topic include a green word? y or n?
n
   index   topic_nr       words   weights
0   2848  topic 337     besvare  0.024165
1   2849  topic 337        lund  0.023099
2   2850  topic 337  udstilling  0.016702
3   2851  topic 337       bente  0.011372
4   2852  topic 337  litteratur  0.009950
5   2853  topic 337    damaskus  0.009240
6   2854  topic 337      forla



Does this topic include a green word? y or n?
n
   index   topic_nr           words   weights
0   2967  topic 350   miljøminister  0.084579
1   2968  topic 350         minkavl  0.039720
2   2969  topic 350         vandløb  0.016822
3   2970  topic 350             wwf  0.014486
4   2971  topic 350         havbrug  0.014019
5   2972  topic 350     friluftsliv  0.012150
6   2973  topic 350        iltsvind  0.011682
7   2974  topic 350           buræg  0.010748
8   2975  topic 350  minkproduktion  0.010280
9   2976  topic 350             fur  0.009346


Does this topic include a green word? y or n?
y

 0 miljøminister 


 1 minkavl 


 2 vandløb 


 3 wwf 


 4 havbrug 


 5 friluftsliv 


 6 iltsvind 


 7 buræg 


 8 minkproduktion 


 9 fur 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:

   index   topic_nr           words   weights
0   2977  topic 351      journalist  0.474522
1   2978  topic 351  socialdemokrat  0.177282
2 



Does this topic include a green word? y or n?
y

 0 udtagning 


 1 lavbundsjorde 


 2 lavbundsjord 


 3 lavbundsjorder 


 4 landdistrikt 


 5 fødevaresektor 


 6 førsteprioritet 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
1 2 3
   index   topic_nr         words   weights
0   3095  topic 364      aktivist  0.095561
1   3096  topic 364  genforhandle  0.048042
2   3097  topic 364  klimaaftaler  0.046997
3   3098  topic 364  sultestrejke  0.044386
4   3099  topic 364          lort  0.039687
5   3100  topic 364    slotsplads  0.027154
6   3101  topic 364    klimasvigt  0.024543
7   3102  topic 364      bullshit  0.024021
8   3103  topic 364         skema  0.023499
9   3104  topic 364     aktivisme  0.022454


Does this topic include a green word? y or n?
y

 0 aktivist 


 1 genforhandle 


 2 klimaaftaler 


 3 sultestrejke 


 4 lort 


 5 slotsplads 


 6 klimasvigt 


 7 bullshit 


 8 skema 


 9 aktivisme 

Are any

In [198]:
len(all_words)

149

In [50]:
temp = []
for word in all_words:
    temp.append(word.lower())
all_words_unique = set(temp)
all_words_unique

{'afgift',
 'biodiversitet',
 'biodiversitetskrise',
 'biogas',
 'biomasse',
 'bæredygtig',
 'bæredygtigt',
 'cirkulærøkonomi',
 'co',
 'coafgift',
 'coneutrale',
 'dkgreen',
 'dkklima',
 'dknatur',
 'dkvind',
 'elbil',
 'elbiler',
 'energi',
 'forbrug',
 'forurening',
 'fossil',
 'fossilbiler',
 'fossile',
 'genanvendelse',
 'grader',
 'grøn',
 'grønne',
 'grønnere',
 'grønomstilling',
 'grønt',
 'havvind',
 'huskklima',
 'klima',
 'klimaaftryk',
 'klimaansvar',
 'klimaet',
 'klimaforandringer',
 'klimaforandringerne',
 'klimakrisen',
 'klimalov',
 'klimaloven',
 'klimamål',
 'klimapolitik',
 'klimatal',
 'klimatopmøde',
 'klimatosse',
 'klimavenlig',
 'klimavenlige',
 'klimavenligt',
 'kloden',
 'kulkraftværker',
 'landbrugspakken',
 'madspild',
 'miljø',
 'miljøet',
 'natur',
 'naturen',
 'oliejagt',
 'oliejagten',
 'olieselskaber',
 'omstilling',
 'opvarmning',
 'parisaftalen',
 'reducere',
 'reduktion',
 'reduktioner',
 'udledning',
 'udledninger',
 'vedvarende',
 'vindmøller',
 '

In [52]:
with open("green_words.txt", "w") as output:
    output.write(str(all_words_unique))

## Finding additional keywords with W2V

In [120]:
with open('green_words.txt', 'r') as f:
    green_words = f.read()

green_words = extract_dict(green_words)

In [1]:
from gensim.models import KeyedVectors

/Users/jeppefoldberg/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
w2v = KeyedVectors.load_word2vec_format('w2v/dsl_skipgram_2020_m5_f500_epoch2_w5.model.w2v.bin', binary=True)

In [56]:
def expand_words(words, n):
    '''takes in a set of words and shows potential words to expand the set'''
    extra_words = []
    for word in words:
        try:
            similar_words = [word[0] for word in w2v.most_similar(positive=word, topn=n) if word[0] not in words]
        except:
            continue
        # also removes words that are already in extra words!
        similar_words = [word for word in similar_words if word not in extra_words]
        for i, word in enumerate(similar_words):
            print('\n', i, word, '\n')
        inp = input('Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:\n')
        indices = [int(i) for i in inp.split(' ') if i.isdigit()]
        for i in indices:
            extra_words.append(similar_words[i])
    return extra_words

In [57]:
extra_words = expand_words(green_words, 10)


 0 klimakrise 


 1 klimaproblemet 


 2 klimaudfordringen 


 3 klimaproblemerne 


 4 klimaproblemer 


 5 biodiversitetskrisen 


 6 klimaudfordringerne 


 7 klimakatastrofen 


 8 klimaudfordringer 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
1 2 3 4 5 6 7 8

 0 c3planterne 


 1 polnære 


 2 frekvenskomponenter 


 3 meridianbuer 


 4 yderteltet 


 5 nattilstand 


 6 antenneanalysator 


 7 crisprteknologi 


 8 hartkornsansættelser 


 9 erstatningskommission 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:


 0 opvarmningen 


 1 opvarming 


 2 drivhuseffekten 


 3 klimaændringer 


 4 drivhuseffekt 


 5 klimaændringerne 


 6 opvarmnings 


 7 temperaturstigninger 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
3 4 5 2 7

 0 gasjagt 


 1 olieefterforskningen 


 2 efterforskningsboringer 


 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
5

 0 klimalovgivning 


 1 klimatraktat 


 2 energilov 


 3 københavneraftale 


 4 immigrationsreform 


 5 energipakke 


 6 klimaplan 


 7 kyotoaftale 


 8 finansreform 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
0 1 3 6 7

 0 afgiften 


 1 afgifter 


 2 miljøafgift 


 3 energiafgift 


 4 ekstraafgift 


 5 elafgift 


 6 co2afgift 


 7 statsafgift 


 8 registreringsafgift 


 9 tillægsafgift 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
2 6

 0 klimamålsætninger 


 1 klimamålsætning 


 2 reduktionsmål 


 3 co2reduktionsmål 


 4 eumål 


 5 klimaplaner 


 6 miljømål 


 7 klimaforpligtelser 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
0 1 2 3 5 7

 0 energiafgift 


 1 co2afgiften 


 0 hybridbil 


 1 ampera 


 2 brintbil 


 3 prius 


 4 hybridbilen 


 5 pluginhybrid 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:


 0 ørkenklima 


 1 kontinentalklima 


 2 kystklima 


 3 klimaforhold 


 4 klimaklassifikation 


 5 köppens 


 6 subarktisk 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:


 0 kildesortering 


 1 genanvende 


 2 genanvendelsen 


 3 genanvendes 


 4 genanvendeligt 


 5 husholdningsaffaldet 


 6 genbrug 


 7 husholdningsaffald 


 8 recirkulering 


 9 pvcaffald 

Are any of these words related to the green debate? Input index numbers of the word seperated by spaces:
1 2 3 4 6 8

 0 olieefterforskning 


 1 gasefterforskning 


 2 olierekord 


 3 gaseventyr 


 4 rederigigant 


 5 gasprojekter 


 6 gasefterforskningen 


 7 gasplatforme 

Are any of these words related to the green debate? Input index numbers of the w

In [59]:
len(extra_words)

160

In [64]:
all_words_combined = extra_words + list(green_words)

In [68]:
all_words_combined = set(all_words_combined)

## Adding keywords from hSBM and W2V to the keywords from qualitative methods 

In [204]:
former_words = pd.read_csv('final_keywords.csv', index_col=0)

In [207]:
former_words

,0
0,olieefterforskningen
1,klimakrisen
2,klimamålene
3,lavemissionsbiler
4,Miljøstyrelsen
...,...
341,klima
342,recirkulering
343,oliejagt
344,2030


In [208]:
former_words_set = set(former_words['0'].values)

In [209]:
all_words_set = set(all_words)

In [212]:
# more new words
all_words_set - former_words_set

{'70',
 'affaldsindsamling',
 'bilafgift',
 'bilfri',
 'biobrændstof',
 'bæredygtighedskrav',
 'c02',
 'ccs',
 'cirkulære',
 'co2e',
 'delmål',
 'dgsb',
 'drivhusgas',
 'drivhusgasudledning',
 'emission',
 'energiø',
 'energiøen',
 'energiøerne',
 'flyafgifter',
 'forurener',
 'greenpeace',
 'greenwashing',
 'havvindmølle',
 'havvindmøllepark',
 'hockeystav',
 'klim',
 'klimaafgifter',
 'klimaaftaler',
 'klimaaktivister',
 'klimaambitioner',
 'klimaambitionerne',
 'klimabelastende',
 'klimabevægelsen',
 'klimabistand',
 'klimaborgerting',
 'klimaborgertinget',
 'klimadebatten',
 'klimaeffekt',
 'klimaflygtninge',
 'klimafond',
 'klimaforhandling',
 'klimaforsker',
 'klimagevinst',
 'klimahandling',
 'klimaindsatsen',
 'klimakompensation',
 'klimalederskab',
 'klimaløsninger',
 'klimaminister',
 'klimamæssigt',
 'klimanøl',
 'klimaområde',
 'klimaomstillingen',
 'klimaordfører',
 'klimapanel',
 'klimapartnerskaberne',
 'klimapartnerskabet',
 'klimapolitisk',
 'klimapotentiale',
 'klimap

In [214]:
all_words_final = list(all_words_set) + list(former_words_set)

In [215]:
all_final_words = pd.DataFrame({'words': all_words_final})

In [217]:
all_final_words.to_csv('final_final_keywords.csv')